In [ ]:
gameFolder = "DOA++-MAME"
#gameFolder = "SFIII-MAME"
#gameFolder = "UMK3-MAME"
#gameFolder = "TEKTAG-MAME"

In [ ]:
import sys, os
import random

sys.path.append(os.path.join(os.path.abspath(''), '../../games',gameFolder))   

from diambraMameGym import *

In [ ]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../customMAME/"
diambraKwargs["frame_ratio"] = 1

# P1
diambraKwargs["player"] = "Random"
# P2
#diambraKwargs["player"] = "P1P2"

In [ ]:
# DOA
diambraKwargs["characters"] =["Random", "Random"]
diambraKwargs["charOutfits"] =[4, 4]

In [ ]:
env = diambraMame("Test", diambraKwargs)

observation = env.reset()

In [ ]:
while True:

    action = env.action_space.sample()
    print("Action1P = ", action)
    if diambraKwargs["player"] == "P1P2":
        action2 = env.action_space.sample()
        action = np.append(action, action2[0])
        action = np.append(action, action2[1])
        print("Action2P = ", action2)
    print("Action = ", action)
    
    observation, reward, done, info = env.step(action)

    print("Frames shape:", observation.shape)
    #print("Reward:", reward)
    #print("Fighting = ", info["fighting"])
    print("Rewards = ", info["rewards"])
    print("HealthP1 = ", info["healthP1"])
    print("HealthP2 = ", info["healthP2"])
    print("PositionP1 = ", info["positionP1"])
    print("PositionP2 = ", info["positionP2"])
    print("WinP1 = ", info["winsP1"])
    print("WinP2 = ", info["winsP2"])
    print("Done = ", done)

    if done:
        print("Resetting Env")
        observation = env.reset()
        
env.close()